In [308]:
import os
import re
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [3]:
#General variables

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

['DWF', 'WWF1', 'WWF2', 'WWF3', 'WWF4', 'WWF5']

In [277]:
#Model specific variables
model_area = "NSSA"

rolling_average = False

#Only for debug then remove
gauges = ['HL14']

calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\SYSTEM_ASSESSMENT\DATA\RAINFALL\NSSA_Rainfall_Data_PDT.dfs0"

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

174    DWF calibration is further improved before WWF...
Name: WWF Text, dtype: object

In [284]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]
report_text


,Zone,DWF Text,WWF Text,Issue Text,Recommendation Text,Model
172,FA2,The flow measurement is spiky. After DWF cali...,FA2 shows good match in total volume and peak ...,Manhole spilling is shown in simulation but ca...,check with local municipality about manhole sp...,NSSA-E
173,LY7,NaN,LY7 combines the flows in LY1 and LY2. The num...,It was found that the previously described con...,The pump setup in the model to match the contr...,NSSA-E
174,MK20,"Due to the lack of reliability with the MK20, ...",DWF calibration is further improved before WWF...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,"Recommend that metering be repeated, at a loca...",NSSA-E
175,MK3,"Due to the lack of reliability with the MK20, ...",MK3 is calibrated together with NV8.\nIn valid...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,Recommend that either the MV MK3 be fixed and ...,NSSA-E
176,NV1,"After DWF calibration, peak level difference a...",NV1 is inconsistent with upstream and downstre...,NaN,NaN,NSSA-E
178,NVLS5,NV19 was used to calibrate for DWF. The per ca...,NVLS1 is located at the outlet chamber. The to...,NVLS5 has no level data.,NaN,NSSA-E
179,NV13,NaN,NV13 has a good match for the October and the ...,NV13 has no data for the two last events.,NaN,NSSA-E
180,NV19,The per capita and per area loading rates have...,NV19 is not calibrated in WWF.,The larger discrepancy between the peak flow r...,NaN,NSSA-E
181,NV20,The per capita and per area loading rates have...,NV20 has a good match in total volumes and pea...,The flow proportionality between NV20 and its ...,"For future metering plans, implementation and ...",NSSA-E
182,NV21,"In terms of field measurements, ADWF of the u/...",NV21 is downstream of New Lynn Siphon and is u...,ADWF of the u/s NV19 is 85.9 L/s which is larg...,Tributary to NV19 and NV21 may benefit from fu...,NSSA-E


In [148]:
gauges


,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node


In [154]:
level_pipes = gauges[gauges['Level Type'].str.contains('Link')][['Node1 (Or Pipe if pipe level)','Level Type']].copy()
level_pipes.rename(columns={'Node1 (Or Pipe if pipe level)':'Node'},inplace=True)
level_pipes

,Node,Level Type
241,43987,Link(DS)
253,54852,Link(DS)
254,54850,Link(DS)


In [155]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)

In [140]:
for ds_level_pipe in ds_level_pipes:
    print(ds_level_pipe)

54850
43987
54852


In [8]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'HL14_20180614_20190403_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

,HL14_Flow_L,HL14_Level
2018-06-14 00:00:00,105.150002,-0.659200
2018-06-14 00:05:00,106.050003,-0.657905
2018-06-14 00:10:00,108.650002,-0.654010
2018-06-14 00:15:00,110.349998,-0.650690
2018-06-14 00:20:00,110.150002,-0.650770
...,...,...
2019-04-03 23:35:00,109.550003,-0.649750
2019-04-03 23:40:00,110.150002,-0.648520
2019-04-03 23:45:00,109.849998,-0.648040
2019-04-03 23:50:00,108.699997,-0.649165


In [9]:
f = 'HL14_20180614_20190403_5min.dfs0'

ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
gauge_id = re.split('_|.',f)[0]
ts['Gauge'] = gauge_id
ts.rename(columns={ts.columns[1]:'Flow'},inplace=True)
if len(ts.columns)>6:
    ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
else:
    ts['Level'] = np.nan
ts = ts[['Gauge','Flow','Level']]


In [279]:
rainfall

,VW14,DN53,VN52,VW49,DN25,DN82,DN54,DN65,VW67,DN72
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2022-01-17 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [278]:
#Import rain gauges

rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            gauge_id = re.split(r'_|[.]',f)[0]
            ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
            ts['Gauge'] = gauge_id
            gauge_ids.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if len(ts.columns)>5:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
            else:
                ts['Level'] = np.nan
            ts = ts[['Gauge','Flow','Level']]
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
            
        

In [161]:
nodes = resultdata.resultData.Nodes
for i, node in enumerate(nodes):
    rid = node.Id


    if rid in level_nodes:
        print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
        for di in node.DataItems:
            if di.Quantity.Id.lower() == 'waterlevel':

nodes

Importing node 10497 at 2022-10-18 14:33:47.234093
waterlevel
Importing node 10498 at 2022-10-18 14:33:47.234093
waterlevel
Importing node 1_119 at 2022-10-18 14:33:47.234093
waterlevel
Importing node 6071 at 2022-10-18 14:33:47.234093
waterlevel
Importing node 6124 at 2022-10-18 14:33:47.234093
waterlevel
Importing node 6819 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 6822 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 7133 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 7169 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 7317 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9488 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9525 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9551 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9561 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9600 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9619 at 2022-10-18 14:33:47.236095
waterlevel
Importing node 9624 a

In [200]:
flow_pipes

{'40472',
 '41250',
 '41295',
 '41304',
 '43611',
 '43648',
 '43945',
 '43946',
 '43953',
 '43987',
 '43989',
 '43999',
 '47391',
 '47438',
 '49559',
 '49580',
 '49582',
 '49592',
 '49640',
 '49652',
 '49709',
 '49727',
 '50870',
 '50924',
 '50932',
 '50933',
 '50940',
 '50952',
 '54287',
 '54850',
 '54852',
 '56296',
 '56393',
 '56394_3',
 '56401',
 '56405',
 'Link_15',
 'Lynn-Total',
 'MKO_Overflow'}

In [252]:
results = []
first_round = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches
        nodes = resultdata.resultData.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

#         missing_muids = set()

#         #Check for missing MUIDs in the result file
#         muid_checks = set()

#         rid_checks = set()
#         for i, reach in enumerate(reaches):
#             rid_checks.add(reach.Id[:(-1 * len(str(i)) - 1)])
#         for rid in network_import_specs["MUID_Extended"].values:
#             if not rid in rid_checks:
#                 muid_checks.add(rid)
#         if len(muid_checks)>0:
#             print('In Network_Import_Specs but not in re: ' + str(muid_checks))
#             break

        for i, node in enumerate(nodes):
            
            rid = node.Id
            if rid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                for di in node.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        values = list(di.CreateTimeSeriesData(0))
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_round == True:
                            level_df_all = flow_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_round = False
        
        
        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count

            if rid in flow_pipes:
                print ("Importing pipe " + reach.Id + " at " + str(dt.datetime.now()))
                print (gridpoints)
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'discharge':

                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        flow_df['MUID'] = rid
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

                        flow_df['DateTimeRef'] = flow_df.index

                        if first_round == True:
                            flow_df_all = flow_df.copy()

                        else:
                            flow_df_all = pd.concat([flow_df_all,flow_df])
                        first_round = False
            
#         for i, reach in enumerate(reaches):
#             rid = reach.Id[:(-1 * len(str(i)) - 1)]
#             gridpoints = reach.GridPoints.Count
#             if rid in flow_pipes:
#                 print ("Importing pipe " + reach.Id + " at " + str(dt.datetime.now()))
#                 for di in reach.DataItems:
#                     if di.Quantity.Id.lower() == 'discharge':

#                         values = list(di.CreateTimeSeriesData(0))
#                         flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
#                         flow_df['ResultFile'] = f
# #                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
#                         flow_df['MUID'] = rid
#                         flow_df['Discharge'] = values                          
#                         flow_df['Discharge'] = flow_df['Discharge'] * 1000

#                         if rolling_average == True:
#                             flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

#                         flow_df['DateTimeRef'] = flow_df.index

#                         if first_round == True:
#                             flow_df_all = flow_df.copy()

#                         else:
#                             flow_df_all = pd.concat([flow_df_all,flow_df])
#                         first_round = False

        first_round = False


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-10-18 15:59:09.107803
Importing node 10497 at 2022-10-18 15:59:09.107803
Importing node 10498 at 2022-10-18 15:59:09.112808
Importing node 1_119 at 2022-10-18 15:59:09.116811
Importing node 6071 at 2022-10-18 15:59:09.120815
Importing node 6124 at 2022-10-18 15:59:09.124819
Importing node 6819 at 2022-10-18 15:59:09.129823
Importing node 6822 at 2022-10-18 15:59:09.132826
Importing node 7133 at 2022-10-18 15:59:09.136830
Importing node 7169 at 2022-10-18 15:59:09.140833
Importing node 7317 at 2022-10-18 15:59:09.144837
Importing node 9488 at 2022-10-18 15:59:09.147840
Importing node 9525 at 2022-10-18 15:59:09.151843
Importing node 9551 at 2022-10-18 15:59:09.154846
Importing node 9561 at 2022-10-18 15:59:09.158850
Importing node 9600 at 2022-10-18 15:59:09.161853
Importing node 9619 at 2022-10-18 15:59:09.165856
Importing node 9624 at 2022-10-18 15:59:09.168859
Importing node 9629 at 2022-10-18 15:59:09.172863
Importing node

Importing pipe 49582-903 at 2022-10-18 15:59:11.063592
3
Importing pipe 49592-913 at 2022-10-18 15:59:11.068597
3
Importing pipe 49640-950 at 2022-10-18 15:59:11.073602
5
Importing pipe 49652-962 at 2022-10-18 15:59:11.078606
3
Importing pipe 49709-1018 at 2022-10-18 15:59:11.083611
5
Importing pipe 49727-1036 at 2022-10-18 15:59:11.088616
9
Importing pipe 50870-1397 at 2022-10-18 15:59:11.094621
5
Importing pipe 50924-1435 at 2022-10-18 15:59:11.099625
3
Importing pipe 50932-1443 at 2022-10-18 15:59:11.104630
3
Importing pipe 50933-1444 at 2022-10-18 15:59:11.109635
3
Importing pipe 50940-1451 at 2022-10-18 15:59:11.113638
3
Importing pipe 50952-1463 at 2022-10-18 15:59:11.118643
3
Importing pipe 54287-1519 at 2022-10-18 15:59:11.123648
3
Importing pipe 54850-1537 at 2022-10-18 15:59:11.128652
3
Importing pipe 54852-1539 at 2022-10-18 15:59:11.133657
3
Importing pipe 56296-1593 at 2022-10-18 15:59:11.138661
3
Importing pipe 56393-1597 at 2022-10-18 15:59:11.143667
7
Importing pipe 563

Importing network NSSA_WWF_Cal_2018-12-12_Base.res1d at 2022-10-18 15:59:13.331668
Importing node 10497 at 2022-10-18 15:59:13.331668
Importing node 10498 at 2022-10-18 15:59:13.337673
Importing node 1_119 at 2022-10-18 15:59:13.342679
Importing node 6071 at 2022-10-18 15:59:13.348684
Importing node 6124 at 2022-10-18 15:59:13.353688
Importing node 6819 at 2022-10-18 15:59:13.359694
Importing node 6822 at 2022-10-18 15:59:13.364699
Importing node 7133 at 2022-10-18 15:59:13.368702
Importing node 7169 at 2022-10-18 15:59:13.373707
Importing node 7317 at 2022-10-18 15:59:13.378711
Importing node 9488 at 2022-10-18 15:59:13.383716
Importing node 9525 at 2022-10-18 15:59:13.388720
Importing node 9551 at 2022-10-18 15:59:13.394726
Importing node 9561 at 2022-10-18 15:59:13.399731
Importing node 9600 at 2022-10-18 15:59:13.404735
Importing node 9619 at 2022-10-18 15:59:13.409739
Importing node 9624 at 2022-10-18 15:59:13.414744
Importing node 9629 at 2022-10-18 15:59:13.419749
Importing node

Importing pipe 41304-129 at 2022-10-18 15:59:15.378541
5
Importing pipe 43611-209 at 2022-10-18 15:59:15.385547
3
Importing pipe 43648-245 at 2022-10-18 15:59:15.392554
7
Importing pipe 43945-350 at 2022-10-18 15:59:15.399561
3
Importing pipe 43946-351 at 2022-10-18 15:59:15.405566
3
Importing pipe 43953-358 at 2022-10-18 15:59:15.412572
5
Importing pipe 43987-390 at 2022-10-18 15:59:15.418578
7
Importing pipe 43989-392 at 2022-10-18 15:59:15.424583
11
Importing pipe 43999-402 at 2022-10-18 15:59:15.431590
7
Importing pipe 47391-580 at 2022-10-18 15:59:15.438596
3
Importing pipe 47438-627 at 2022-10-18 15:59:15.444601
3
Importing pipe 49559-884 at 2022-10-18 15:59:15.451608
3
Importing pipe 49580-901 at 2022-10-18 15:59:15.458614
11
Importing pipe 49582-903 at 2022-10-18 15:59:15.464620
3
Importing pipe 49592-913 at 2022-10-18 15:59:15.471626
3
Importing pipe 49640-950 at 2022-10-18 15:59:15.477632
5
Importing pipe 49652-962 at 2022-10-18 15:59:15.484638
3
Importing pipe 49709-1018 at 

In [243]:
flow_pipes

{'40472',
 '41250',
 '41295',
 '41304',
 '43611',
 '43648',
 '43945',
 '43946',
 '43953',
 '43987',
 '43989',
 '43999',
 '47391',
 '47438',
 '49559',
 '49580',
 '49582',
 '49592',
 '49640',
 '49652',
 '49709',
 '49727',
 '50870',
 '50924',
 '50932',
 '50933',
 '50940',
 '50952',
 '54287',
 '54850',
 '54852',
 '56296',
 '56393',
 '56394_3',
 '56401',
 '56405',
 'Link_15',
 'Lynn-Total',
 'MKO_Overflow'}

In [123]:
import plotly
a = np.random.normal(0,1,100)
b = np.random.normal(-2,5,100)

c = np.random.normal(0,1,100)
d = np.random.normal(-2,5,100)

fig = plotly.tools.make_subplots(rows=2,cols=1)

trace_rm1 = go.Histogram(x = a, opacity = 0.75, name = 'malignant')
trace_rm2 = go.Histogram(x = b, opacity = 0.75, name = 'benign')
fig.append_trace(go.Histogram(x = a, opacity = 0.75, name = 'benign'),1,1)
fig.append_trace(go.Histogram(x = b, opacity = 0.75, name = 'malignant'),1,1)
fig.append_trace(go.Histogram(x = c, opacity = 0.75, name = 'benign'),2,1)
fig.append_trace(go.Histogram(x = d, opacity = 0.75, name = 'malignant'),2,1)
fig.layout.update(go.Layout(barmode = 'overlay',))

fig.show()

In [244]:
flow_df_all.MUID.unique()

array(['40472', '41250', '41295', '41304', '43611', '43648', '43945',
       '43946', '43953', '43987', '43989', '43999', '47391', '47438',
       '49559', '49580', '49582', '49592', '49640', '49652', '49709',
       '49727', '50870', '50924', '50932', '50933', '50940', '50952',
       '54287', '54850', '54852', '56296', '56393', '56394_3', '56401',
       '56405', 'Link_15'], dtype=object)

In [251]:
flow_df_all.loc[flow_df_all.MUID==pipe]

,ResultFile,MUID,Discharge,DateTimeRef
2020-01-29 00:00:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,537.4089,2020-01-29 00:00:00
2020-01-29 00:05:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,524.1130,2020-01-29 00:05:00
2020-01-29 00:10:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,525.2472,2020-01-29 00:10:00
2020-01-29 00:15:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,541.1195,2020-01-29 00:15:00
2020-01-29 00:20:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,575.0741,2020-01-29 00:20:00
...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,810.7676,2020-02-01 23:40:00
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,827.4142,2020-02-01 23:45:00
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,841.2172,2020-02-01 23:50:00
2020-02-01 23:55:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,43989,880.8254,2020-02-01 23:55:00


In [348]:
str(list(report_text[report_text.Zone=='MK20']['DWF Text'])[0])

'Due to the lack of reliability with the MK20, MK3 and NV9 meters, these three calibration points had originally been calibrated to NV8.\xa0 This was because both the local and cumulative GWI per area appeared high compared to the majority of the stations.\xa0 In the subsequent WWF calibration, the DWF calibration of MK20 was further modified to gain improvement.\xa0 This is why MK20 now has a different set of DWF parameters when compared to MK3 and NV9.'

In [351]:
#Check here for secondary Y xis title:
#https://chart-studio.plotly.com/~empet/15656/example-of-subplots-with-rows1/#/

import plotly
gauge = 'NV1'
zone = gauge #Change this!!!!!

pipe = str(list(gauges[gauges.Gauge==gauge]['Pipe'])[0])
node = str(list(gauges[gauges.Gauge==gauge]['Node1 (Or Pipe if pipe level)'])[0])
rain_gauge = str(list(gauges[gauges.Gauge==gauge]['Rain Gauge'])[0])


with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
    
    f.write('<h1 style="text-align:center">Model Calibration Report for ' + gauge + ' in ' + model_area + '</h1>')
    
    f.write('<h2>1. Description of the Calibration Area</h2>')
    f.write('<h2>2. Description of the Sensor and Network</h2>')
    f.write('<h2>3. Model Calibration Setup: </h2>')
    f.write('<h2>4. Zone Map</h2>')
    f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">')
    
    f.write('<h2>5. DWF Calibration Summary</h2>')  
    f.write('<p>' + str(list(report_text[report_text.Zone=='MK20']['DWF Text'])[0]) + '</p>')
    
    f.write('<h2>6. WWF Calibration and Validation Summary</h2>')
    f.write('<p>' + str(list(report_text[report_text.Zone=='MK20']['WWF Text'])[0]) + '</p>')
    
    f.write('<h2>7. Calibration Issues</h2>')
    f.write('<p>' + str(list(report_text[report_text.Zone=='MK20']['Issue Text'])[0]) + '</p>')
    
    f.write('<h2>8. Recommendations</h2>')
    f.write('<p>' + str(list(report_text[report_text.Zone=='MK20']['Recommendation Text'])[0]) + '</p>')
    
    f.write('<h2>9. DWF Calibration Plots</h2>')
    
#     f.write('<h2>4. Zone Map</h2>')
    
#     <img src="img_chania.jpg" alt="Flowers in Chania">
    

    for period_spec in period_specs:
        start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
        end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

        chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

        measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)]

        measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

        result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)]

        result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

    #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]

        fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=('Discharge',  'Water Level'),specs=[[{"secondary_y": True}, {"secondary_y": True}]])
    #     fig = make_subplots(specs=[[{'secondary_y': True}]])

        fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
        fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
        fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)


        fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level'),1,2)
        fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level'),1,2)

        fig.update_layout(
            autosize=False,
            width = 1600,
            height=250,
            margin=dict(
                l=0,
                r=0,
                b=25,
                t=60,
                pad=4
                ),
    #         yaxis_title="Dicharge (L/s)",
            title ={
                'text' : chart_header,
                'x':0.45,
                'xanchor': 'center'
            })

        fig['layout']['yaxis']['title']='Dicharge (L/s)'
        fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
    #     fig['layout']['yaxis2']['autorange']='reversed'
        fig['layout']['yaxis2']['range']=[10,0]
        fig['layout']['yaxis3']['title']='Water Level (m)'



    #     fig.update_yaxes(rangemode="tozero")    

        fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


    #             f.write('<h1>Long Term Simulation ' + w1.value + '</h1>')
    #             f.write('<h2>Model Result</h1>')

f.close()
    
    

In [303]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
from plotly.subplots import make_subplots
np.random.seed(123)

fig = make_subplots(specs=[[{'secondary_y': True}]])

n=8

d = {'date': [datetime(2020, 3, k ) for k in range(2, n+2)],
     'A': 21+12*np.random.rand(n),
     'B': 16+12*np.random.rand(n),
     'C': 14+10*np.random.rand(n)}
df = pd.DataFrame(d)

fig.add_trace(go.Scatter(x=df['date'], y= df['A']), secondary_y=False)  
fig.add_trace(go.Scatter(x=df['date'], y= df['B']), secondary_y=False)  

fig.add_trace(go.Scatter(x=df['date'], y= df['C']), secondary_y=True) 


In [20]:
gauges
    

,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node
